In [ ]:
from selenium import webdriver
import requests
import time
import re
from bs4 import BeautifulSoup
import pickle
import pandas as pd

In [ ]:
# selenium version
# def parse_content():
#     '''글에 있는 내용을 스크래핑하는 함수
    
#     output: page_index, title, date, question, answer, office를 저장한 딕셔너리'''
    
#     return {"title":driver.find_elements_by_css_selector("#Form th.tit")[0].text,
#             "date":driver.find_elements_by_css_selector("#Form .date")[0].text,
#             "question":driver.find_elements_by_css_selector(".questionDiv tbody div")[0].text,
#             "answer":driver.find_elements_by_css_selector(".answerTxt")[0].text,
#             "office":driver.find_elements_by_css_selector(".civilPart .pl10")[0].text}

In [ ]:
# selenium version
# if __name__ == "__main__":
#     driver = webdriver.Chrome()
#     driver.get("https://www.epeople.go.kr/jsp/user/pc/policy/uPcOpenCivilList.paid")

#     contents = []
#     postList = []

#     i = 1

#     try:
#         while True:
#             # 해당 페이지로 이동하는 자바스크립트 함수를 실행한다.
#             driver.execute_script("javascript:gotoPage({})".format(i))

#             # 현재 페이지에서 볼 수 있는 글 목록을 모두 리스트에 저장한다.
#             temp = [_.get_attribute("href") for _ in driver.find_elements_by_css_selector(".taL a")]
#             postList.extend(temp)

#             while postList:
#                 # 내용으로 이동한다
#                 driver.execute_script(postList.pop(0))

#                 # 제목, 날짜, 질문내용, 답변내용, 담당부서를 딕셔너리형태로 리스트에 추가한다.
#                 contents.append(parse_content())
#                 # 글 목록으로 돌아간다.
#                 driver.back()

#             # 다음 페이지로 넘어가도록 i를 증가시킨다.
#             i += 1
#             if i % 100 == 0:
#                 print(i)
#     except:
#         print("crawling finished")

In [ ]:
headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}

def download(method, url, param=None, data=None, timeout=1, maxretries=3):
    '''
    request 패키지를 이용해서 error handling
    '''
    try:
        resp = requests.request(method, url,params=param, data=data, headers=headers)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and maxretries > 0:
            print(maxretries)
            time.sleep(timeout)
            download(method, url, param, data, timeout, maxretries-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp

In [ ]:
def find_contents_num():
    '''국민 신문고에 1년치 민원의 개수를 리턴
    
    output:
    maximum: 민원 개수
    '''
    
    url = "https://www.epeople.go.kr/jsp/user/pc/policy/uPcOpenCivilList.paid"
    html = download("get", url)
    dom = BeautifulSoup(html.text)
    maximum = int(dom.select_one(".page span").text)
    
    return maximum

In [ ]:
# BeautifulSoup version
def get_href(url, pageNum):
    ''' 국민신문고 민원 목록 페이지에서 글로 가는 링크를 찾아오는 함수
    
    input:
        url: 크롤링할 url
        pageNum: 글 목록에 있는 링크를 가져올 
    
    output:
        links: 글로 가는 링크(href)를 담은 리스트
        hits: 조회수
    '''
    
    # 민원 목록 페이지를 html로 parsing
    html = download("get", url, {"pageNum":pageNum})
    dom = BeautifulSoup(html.text)
    
    lists = dom.select(".listForm1.mt10 tbody tr")
    
    # 글로 가는 링크를 리스트에 넣는다.
    links = [_.select_one("a")["href"] for _ in lists]
            
    # 조회수는 글 목록에만 있으므로 여기서 찾는다
    hits = [_.select("td")[-1].text.strip() for _ in lists]
            
    return (links, hits)

In [ ]:
# BeautifulSoup version
def parse_content(url, params, hits):
    ''' url과 params를 받아서 해당 글에 있는 민원 정보를 받아오는 함수
    
    input:
        url: 글 내용 url
        params: 해당 글 내용을 받아올 파라미터
        hits: 조회수를 담은 리스트, 조회수는 글 내용이 아니라 목록에서 확인할 수 있으므로 파라미터로 넘겨준다.
        
    output:
        딕셔너리: 목록번호, 제목, 등록일, 질문내용, 답변내용, 담당부서, 조회수가 저장돼 있다. 
    '''
    
    # 해당 글에 있는 html을 parsing
    html = download("get", url, params)
    dom = BeautifulSoup(html.text)
    
    return {'목록 번호': params["faq_no_n"],
             '제목':dom.select_one("#Form th.tit").text,
             '등록일':dom.select_one("#Form .date").text,
             '질문내용':dom.select_one(".questionDiv tbody div").text.strip(),
             '답변내용':dom.select_one(".answerTxt").text,
             '담당부서':dom.select(".civilPart .pl10")[-1].text,
             '조회수': hits}

In [ ]:
data = pd.read_csv("sinmungo.csv")

In [ ]:
seen = data.page_index.tolist()
maximum = find_contents_num() // 10 + 1

# 민원 목록 url
baseURL = "https://www.epeople.go.kr/jsp/user/pc/policy/uPcOpenCivilList.paid"

# 민원 상세 내용 url
contentURL = "https://www.epeople.go.kr/jsp/user/pc/policy/UPcUnionPolicyDetail.paid"
contents= []

# 국민 신문고에서 민원 상세 내용으로 넘겨주는 하이퍼링크는 javacript fucntion으로 작성돼있다.
# 해당 함수에서 필요로 하는 파라미터만 갖고 와서 get 방식에 넘겨줄 파라미터를 만들어서 html을 파싱한다.
# 매개 변수가 없어도 안 넣어주면 오류나서 모두 넣어준다.
params = {
    "sdetail":"1",
    "strFlag":"",
    "strArea":"",
    "strBody":"",
    "cat_name":"",
    "strFrom_ex":"",
    "strTo_ex":"",
    "pageNum":"",
    "s_anc_c":"",
    "flag":"3",
    "faq_no_n":None,    # 민원 상세 내용에 따라 달라지는 파라미터
    "civil_no_c":"",
    "peti_no_c":"",
    "strFrom":"",
    "strTo":"",
    "so_gubun1":"",
    "so_gubun2":"",
    "so_gubun3":"",
    "so_gubun4":"",
    "search_gubun1":"",
    "search_gubun2":"",
    "search_gubun3":"",
    "so_recom":"",
    "so_recom_Subanc":"",
    "show_sele":"10"}

In [ ]:
for i in range(8500,maximum):
    links, hits = get_href(baseURL, i+1)
    
    while links:
        temp = (links.pop(0), hits.pop(0))
        
        # 이미 봤던 페이지는 넘어간다.
        if re.findall(r"[\d]+", temp[0])[1] in seen:
            continue
        
        # 봤던 페이지가 아니면 민원 내용을 딕셔너리에 저장한다.
        seen.append(re.findall(r"[\d]+", temp[0])[1])
        params["faq_no_n"] = re.findall(r"[\d]+", temp[0])[1]
        contents.append(parse_content(contentURL, params, temp[1]))
        
        if len(seen) % 1000 == 0:
            print("seen: {}".format(len(seen)))

In [ ]:
data = pd.concat([data.drop_duplicates(), pd.DataFrame.from_dict(contents).drop_duplicates()], ignore_index=True)

In [ ]:
data.to_csv("sinmungo.csv", encoding="utf-8", index=False)